# Differentiable Higher-Order Functions

Higher-order functions are functions that operate on other functions—either by taking them as arguments or by returning them. This notebook explores how to implement differentiable higher-order functions in TensorFlow, enabling gradient-based optimization of function structures themselves.

## What Are Higher-Order Functions?

[Higher-order functions](https://en.wikipedia.org/wiki/Higher-order_function) are fundamental in functional programming and enable:
- **Function composition**: Building complex functions from simpler ones
- **Currying**: Converting multi-argument functions into chains of single-argument functions
- **Function factories**: Generating specialized functions based on parameters
- **Meta-programming**: Programs that manipulate or generate other programs

## Why Make Them Differentiable?

By making higher-order functions differentiable, we can:
- **Learn function structures**: Optimize not just parameters but the function form itself
- **Automatic function discovery**: Let gradient descent find optimal function architectures
- **Meta-learning**: Train systems that learn to generate appropriate functions for tasks
- **Compositional learning**: Build complex behaviors from learned components

This bridges traditional functional programming with modern neural network optimization.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Setup

Let's import the necessary libraries:

In [2]:
%matplotlib inline

In [ ]:
## Curried Functions: Building Specialized Functions

[Currying](https://stackoverflow.com/questions/36314/what-is-currying) is a technique of converting a function with multiple arguments into a sequence of functions, each with a single argument.

### Mathematical Concept

Instead of: $f(x_1, x_2) = x_1 \times x_2$

We create: $g(x_1) = h$ where $h(x_2) = x_1 \times x_2$

### Differentiability Considerations

**Key insight**: TensorFlow 2.x has limitations with `@tf.function` on functions that don't return tensors. Therefore:
- **Outer function** (the generator): Executes eagerly
- **Inner function** (the returned function): Executes in graph mode with full differentiability

This hybrid approach maintains gradient flow while working within TensorFlow's constraints.

### Benefits

- **Partial application**: Fix some arguments while keeping others variable
- **Function specialization**: Create optimized versions for specific use cases
- **Composable design**: Build complex functions from simpler curried components

## Simple Example of a curried function

[StackOverflow Example](https://stackoverflow.com/questions/36314/what-is-currying)

We take two variables $x1$ and $x2$ and multiply them together. We can see that tensorflow is able to compute the gradient with respect both these variables.

Note: Tensorflow 2.1.0 does not support `@tf.function` for functions that do not return a tensor. This may change in future. So, the outer function is executed eagerly, while the inner function is in graph mode.

### Implementing a Curried Function

The `curried_generator` demonstrates the basic pattern:

## Learnable Polynomial Functions

Now let's explore a more sophisticated example: **learning polynomial functions**. We'll create a higher-order function that generates polynomials based on learnable parameters.

### Polynomial Representation

We encode polynomials using a coefficient-exponent matrix representation:

**Example**: $f(x) = 4x^2 + 3x^3 + 2x^5$

$$\text{definition} = \begin{bmatrix}
\text{coefficient} & \text{exponent} \\
4.0 & 2.0 \\
3.0 & 3.0 \\
2.0 & 5.0
\end{bmatrix}$$

### Advantages of This Representation

1. **Flexible structure**: Can represent polynomials of any degree
2. **Learnable parameters**: Both coefficients and exponents can be optimized
3. **Compositional**: Easy to add/remove terms during learning
4. **Differentiable**: All operations preserve gradient flow

This enables **automatic function discovery** where the system learns not just the coefficients but the entire polynomial structure!

In [4]:
def curried_generator(x1):
    @tf.function
    def inner(x2):
        return x1 * x2
    return inner

### Polynomial Generator Implementation

The `polynomial_generator` creates a differentiable function from a parameter matrix:

### Testing Gradient Flow

Let's verify that gradients flow through both the input variables and the function definition:

The polynomial $f(x) = 2x + 3x^2 + x^4$ evaluated at $x = [1,2,3]$ produces gradients showing:
- **w.r.t. x**: The derivative of the polynomial at each point
- **w.r.t. definition**: How changing coefficients/exponents affects the output

This dual differentiability is crucial for learning function structures!

## Polynomial Learning Experiment

Let's demonstrate **automatic function discovery** by training a system to learn a target polynomial from data alone.

### Target Function

We'll generate training data from the polynomial:
$$f(x) = 2x - 3x^2 + x^4$$

The challenge: Can our system discover this exact functional form by observing input-output pairs?

### Experimental Setup

1. **Ground truth**: Generate data from the known polynomial
2. **Unknown model**: Initialize with random coefficients and exponents  
3. **Learning objective**: Minimize prediction error on the data
4. **Discovery**: Let gradient descent find the optimal function structure

In [5]:
x1 = tf.Variable([2,2,2], dtype=tf.float32)
x2 = tf.Variable([1,2,3], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    eqn = curried_generator(x1)
    z = eqn(x2)
    
print(eqn)
print(tape.gradient(z, x1))
print(tape.gradient(z, x2))

tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)
tf.Tensor([2. 2. 2.], shape=(3,), dtype=float32)


### Ground Truth Function

Let's define and visualize our target polynomial:

## Training the Function Discovery System

Now we initialize a learnable polynomial with random parameters and train it to fit our target function.

### Training Process

1. **Random initialization**: Start with completely random coefficients and exponents
2. **Function generation**: Create a polynomial function from current parameters
3. **Prediction**: Evaluate the generated function on training inputs
4. **Loss computation**: Compare predictions with ground truth using L2 loss
5. **Optimization**: Update parameters using Adam optimizer
6. **Iteration**: Repeat until convergence

### Key Innovation

Unlike traditional polynomial fitting (which only learns coefficients), our system learns **both coefficients and exponents**, enabling discovery of the complete functional form!

## Polynomial Regression

Below is an example of a generator function, which takes a function definition as an argument and returns a function which has the same tensorflow graph as the polynomial. 

The equation is encoded as follows

$$
equation = 4x^2 + 3x ^ 3 + 2x ^ 5
$$

\begin{equation*}
definition = 
\begin{bmatrix}
constant & exponent \\
4.0 & 2.0 \\
3.0 & 3.0 \\
2.0 & 5.0 \\
\end{bmatrix}\end{equation*}

### Training Implementation

The training step demonstrates the complete learning pipeline:

## Results: Successful Function Discovery

The learning system has successfully discovered a function that closely approximates our target polynomial!

In [6]:
def polynomial_generator(definition):
    definition = tf.unstack(definition)
    
    @tf.function
    def inner(x):
        result = 0.0
        for term in definition:
            constant = term[0]
            exponent = term[1]
            result += constant * tf.pow(x, exponent)
        return result
    return inner

### Visual Comparison

The plot shows remarkable agreement between the fitted (red dashed) and actual (blue dotted) functions, demonstrating successful function discovery.

### Function Form Comparison

**Original target**: `+ 2.00 * x ^ 1.00  -3.00 * x ^ 2.00 + 1.00 * x ^ 4.00`  
**Learned function**: `+ 1.10 * x ^ 0.74 + 0.59 * x ^ 4.63  -1.68 * x ^ 2.15`

### Key Insights

1. **Alternative representation**: The system found a different polynomial that produces nearly identical outputs
2. **Functional equivalence**: Despite different parameters, both functions approximate the same curve
3. **Creative solution**: The learner discovered that $1.10x^{0.74} + 0.59x^{4.63} - 1.68x^{2.15}$ behaves similarly to $2x - 3x^2 + x^4$ in the training domain

This demonstrates the power of differentiable higher-order functions: the system can discover novel functional forms that achieve the desired behavior!

## Summary

We successfully implemented differentiable higher-order functions that enable:

### Key Achievements

- ✅ **Curried functions**: Maintained differentiability across function closures
- ✅ **Function generators**: Created parametric function families
- ✅ **Automatic discovery**: Learned both coefficients and exponents simultaneously  
- ✅ **Alternative forms**: Found equivalent representations of target functions

### Applications

This framework enables:
- **Neural architecture search**: Learning optimal network structures
- **Symbolic regression**: Discovering mathematical relationships in data
- **Meta-learning**: Functions that learn to generate other functions
- **Compositional AI**: Building complex behaviors from learned components

Differentiable higher-order functions represent a powerful bridge between traditional functional programming and modern deep learning, enabling systems that can discover and optimize their own computational structures.

In [7]:
definition = tf.Variable([
    [2.0, 1.0],
    [3.0, 2.0],
    [1.0, 4.0]
], dtype=tf.float32) # 2 * x + 3 * x ^ 2 + 1 * x ^ 4
x = tf.Variable([1,2,3], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    eqn = polynomial_generator(definition)
    z = eqn(x)
    
print(tape.gradient(z, x))
print(tape.gradient(z, definition))

tf.Tensor([ 12.  46. 128.], shape=(3,), dtype=float32)
tf.Tensor(
[[  6.         9.364263]
 [ 14.        37.9803  ]
 [ 98.       100.07795 ]], shape=(3, 2), dtype=float32)


In [8]:
def pretty_print_definition(definition):
    definition = tf.unstack(definition)
    result = ''
    template = "{} {:.2f} * x ^ {:.2f} "
    for term in definition:
        constant = np.round(term[0].numpy(), 2)
        exponent = np.round(term[1].numpy(), 2)
        sign_separator = '+' if constant >= 0 else ''
        result += template.format(sign_separator, constant, exponent)
    return result
tf.print(definition)
pretty_print_definition(definition)

[[2 1]
 [3 2]
 [1 4]]


'+ 2.00 * x ^ 1.00 + 3.00 * x ^ 2.00 + 1.00 * x ^ 4.00 '

### Generating data

To illustrate that the model can learn to fit a polynomial, we generate data for ground truth.

We sample $100$ points uniformly between $0.0$ and $1.5$ for the equation below.

$$
equation = 2x - 3x ^ 2 + x ^ 4
$$

In [9]:
definition = tf.Variable([
    [2.0, 1.0],
    [-3.0, 2.0],
    [1.0, 4.0]
], dtype=tf.float32) # 2 * x - 3 * x ^ 2 + 1 * x ^ 4
original_definition = definition # Take a backup
pretty_print_definition(definition)

'+ 2.00 * x ^ 1.00  -3.00 * x ^ 2.00 + 1.00 * x ^ 4.00 '

In [10]:
eqn = polynomial_generator(definition)
xs = tf.random.uniform((100,), dtype=tf.float32) * 1.5
ys = eqn(xs)

s_xs = tf.sort(xs)
s_ys = eqn(s_xs)
plt.plot(s_xs, s_ys)
plt.show()

### Training

For training, we initialize a new function definition with uniform random variables.

The loss is the L2 loss between the predicted $y$ and the ground truth $y$

In [11]:
opt = tf.keras.optimizers.Adam()

definition = tf.Variable(tf.random.uniform((3, 2)), dtype=tf.float32)

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        eqn = polynomial_generator(definition) # Generate a new equation with updated definition
        ys_ = eqn(xs) # Predict new y
        loss = tf.nn.l2_loss(ys - ys_) # Find loss
    grads = tape.gradient(loss, definition)
    opt.apply_gradients(zip([grads], [definition])) # Update the polynomial definition
    del eqn # Free memory
    return loss

In [12]:
for i in range(10000):
    loss = train_step()
    if i % 1000 == 0:
        equation_deviation = tf.nn.l2_loss(definition - original_definition)
        tf.print(loss, equation_deviation)

8.33185 13.3747196
2.39628935 14.0850811
1.60698581 12.3505411
0.537338912 12.8752937
0.0742688552 14.2031412
0.0259165019 14.7807331
0.0182985067 14.9476681
0.0137615316 15.1220484
0.0105026718 15.2986126
0.00782978907 15.4724398


### Results

The generated equation varies between runs but it resembles the ground truth equation, closely.

In [13]:
eqn = polynomial_generator(definition)
ns_ys = eqn(s_xs)
plt.plot(s_xs, ns_ys, color='red',linestyle='--', label='fitted')
plt.plot(s_xs, s_ys, color='blue',linestyle='dotted', label='actual')
plt.legend()
plt.show()

In [14]:
print(pretty_print_definition(original_definition))
print(pretty_print_definition(definition))

+ 2.00 * x ^ 1.00  -3.00 * x ^ 2.00 + 1.00 * x ^ 4.00 
+ 1.10 * x ^ 0.74 + 0.59 * x ^ 4.63  -1.68 * x ^ 2.15 
